# agentscope.tool 

## 1. Toolkit 类概述

Toolkit 是 AgentScope 中用于管理工具函数的核心类，支持：
- 函数级别的工具管理
- 分组级别的工具管理
- MCP (Model Context Protocol) 服务器集成
- 动态工具激活/停用

In [1]:
## 一、基础工具注册和使用

from agentscope.tool import Toolkit, execute_python_code, ToolResponse
from agentscope.agent import ReActAgent
from agentscope.model import DashScopeChatModel
from agentscope.formatter import DashScopeChatFormatter
from agentscope.message import Msg
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

# 创建工具包
toolkit = Toolkit()

# 注册内置工具函数
toolkit.register_tool_function(execute_python_code)

# 查看已注册的工具JSON schema
schemas = toolkit.get_json_schemas()
print("已注册的工具:")
for schema in schemas:
    print(f"- {schema['function']['name']}: {schema['function']['description']}")

# 创建带工具的Agent
agent = ReActAgent(
    name="ToolAgent",
    sys_prompt="你是一个能够执行Python代码的助手",
    model=DashScopeChatModel(
        model_name="qwen-max",
        api_key=os.environ["DASHSCOPE_API_KEY"],
        stream=True,
        enable_thinking=True,
    ),
    formatter=DashScopeChatFormatter(),
    toolkit=toolkit,
)

# 测试工具使用
test_msg = Msg(
    name="user",
    content="请帮我计算 1+1 的结果",
    role="user"
)

response = await agent(test_msg)
print(f"Agent回复: {response.content}")

已注册的工具:
- execute_python_code: Execute the given python code in a temp file and capture the return

code, standard output and error. Note you must `print` the output to get
the result, and the tmp file will be removed right after the execution.
ToolAgent: {
    "type": "tool_use",
    "id": "call_3930bd3f70b2450eba7da6",
    "name": "execute_python_code",
    "input": {
        "code": "print(1+1)"
    }
}
system: {
    "type": "tool_result",
    "id": "call_3930bd3f70b2450eba7da6",
    "name": "execute_python_code",
    "output": [
        {
            "type": "text",
            "text": "<returncode>0</returncode><stdout>2\n</stdout><stderr></stderr>"
        }
    ]
}
ToolAgent: 计算 1+1 的结果是 2。
Agent回复: 计算 1+1 的结果是 2。


In [3]:
## 二、自定义工具函数

# 定义自定义工具函数
async def calculate_fibonacci(n: int) -> ToolResponse:
    """
    计算斐波那契数列的第n项
    
    Args:
        n: 要计算的项数（正整数）
    
    Returns:
        ToolResponse: 包含计算结果的响应
    """
    if n <= 0:
        return ToolResponse(
            content=[{"type": "text", "text": "错误：n必须是正整数"}],
            metadata={"error": True}
        )
    
    if n <= 2:
        result = 1
    else:
        a, b = 1, 1
        for _ in range(2, n):
            a, b = b, a + b
        result = b
    
    return ToolResponse(
        content=[{"type": "text", "text": f"斐波那契数列第{n}项的值是: {result}"}],
        metadata={"result": result, "n": n}
    )

# 狼人杀游戏相关工具
async def check_player_role(player_name: str, game_state: dict = None) -> ToolResponse:
    """
    检查玩家的角色信息（仅用于演示）
    
    Args:
        player_name: 玩家姓名
        game_state: 游戏状态字典（可选）
    
    Returns:
        ToolResponse: 包含角色信息的响应
    """
    # 模拟角色数据
    roles = {
        "Alice": "村民",
        "Bob": "狼人", 
        "Charlie": "预言家",
        "Diana": "女巫"
    }
    
    role = roles.get(player_name, "未知角色")
    
    return ToolResponse(
        content=[{"type": "text", "text": f"{player_name}的角色是: {role}"}],
        metadata={"player": player_name, "role": role}
    )

# 先创建工具组，然后再注册工具函数
toolkit.create_tool_group(
    group_name="math",
    description="数学计算相关工具",
    active=True,
    notes="使用数学工具时请确保输入参数的有效性"
)

toolkit.create_tool_group(
    group_name="werewolf", 
    description="狼人杀游戏相关工具",
    active=True,
    notes="这些工具用于狼人杀游戏中的角色和状态查询"
)

# 注册自定义工具到对应的工具组
toolkit.register_tool_function(calculate_fibonacci, group_name="math")
toolkit.register_tool_function(check_player_role, group_name="werewolf")

print("注册自定义工具后的工具列表:")
for schema in toolkit.get_json_schemas():
    print(f"- {schema['function']['name']}")

print(f"\n当前激活的工具组notes:\n{toolkit.get_activated_notes()}")

注册自定义工具后的工具列表:
- execute_python_code
- generate_response
- calculate_fibonacci
- check_player_role

当前激活的工具组notes:
## About math Tools
使用数学工具时请确保输入参数的有效性
## About werewolf Tools
这些工具用于狼人杀游戏中的角色和状态查询


In [5]:
## 三、工具分组管理

# 检查并创建工具分组（避免重复创建）
def create_tool_group_if_not_exists(toolkit, group_name, description, active=True, notes=None):
    """如果工具组不存在则创建"""
    try:
        toolkit.create_tool_group(
            group_name=group_name,
            description=description,
            active=active,
            notes=notes
        )
        print(f"✓ 创建工具组: {group_name}")
    except ValueError as e:
        if "already registered" in str(e):
            print(f"⚠ 工具组 {group_name} 已存在，跳过创建")
            # 更新工具组状态
            toolkit.update_tool_groups([group_name], active=active)
        else:
            raise e

# 创建或更新工具分组
create_tool_group_if_not_exists(
    toolkit,
    group_name="math",
    description="数学计算相关工具",
    active=True,
    notes="使用数学工具时请确保输入参数的有效性"
)

create_tool_group_if_not_exists(
    toolkit,
    group_name="werewolf", 
    description="狼人杀游戏相关工具",
    active=False,  # 默认不激活
    notes="这些工具仅在狼人杀游戏进行时使用"
)

# 查看当前激活的工具组
print("\n当前激活的工具组notes:")
print(toolkit.get_activated_notes())

# 动态激活/停用工具组
print("\n激活狼人杀工具组...")
toolkit.update_tool_groups(["werewolf"], active=True)

print("激活后的工具schemas:")
for schema in toolkit.get_json_schemas():
    func_name = schema['function']['name']
    # 由于 JSON schema 中可能没有 group 字段，我们需要其他方式判断分组
    print(f"- {func_name}")

# 停用数学工具组
print("\n停用数学工具组...")
toolkit.update_tool_groups(["math"], active=False)

print("停用后的激活notes:")
print(toolkit.get_activated_notes())

# 重新激活数学工具组
print("\n重新激活数学工具组...")
toolkit.update_tool_groups(["math"], active=True)

print("最终的工具列表:")
final_schemas = toolkit.get_json_schemas()
for schema in final_schemas:
    print(f"- {schema['function']['name']}: {schema['function']['description']}")

⚠ 工具组 math 已存在，跳过创建
⚠ 工具组 werewolf 已存在，跳过创建

当前激活的工具组notes:
## About math Tools
使用数学工具时请确保输入参数的有效性

激活狼人杀工具组...
激活后的工具schemas:
- execute_python_code
- generate_response
- calculate_fibonacci
- check_player_role

停用数学工具组...
停用后的激活notes:
## About werewolf Tools
这些工具用于狼人杀游戏中的角色和状态查询

重新激活数学工具组...
最终的工具列表:
- execute_python_code: Execute the given python code in a temp file and capture the return

code, standard output and error. Note you must `print` the output to get
the result, and the tmp file will be removed right after the execution.
- generate_response: Generate a response. Note only the input argument `response` is

visible to the others, you should include all the necessary
information in the `response` argument.
- calculate_fibonacci: 计算斐波那契数列的第n项
- check_player_role: 检查玩家的角色信息（仅用于演示）


In [6]:
## 四、预设参数和后处理函数

# 定义带预设参数的工具
async def send_game_message(message: str, channel: str = "general", priority: str = "normal") -> ToolResponse:
    """
    发送游戏消息到指定频道
    
    Args:
        message: 要发送的消息内容
        channel: 消息频道
        priority: 消息优先级
    """
    return ToolResponse(
        content=[{"type": "text", "text": f"消息已发送到{channel}频道: {message} (优先级: {priority})"}],
        metadata={"channel": channel, "message": message, "priority": priority}
    )

# 定义后处理函数
def message_postprocessor(tool_call, tool_response):
    """对发送消息的结果进行后处理"""
    if tool_response.metadata and tool_response.metadata.get("priority") == "high":
        # 高优先级消息添加特殊标记
        new_content = tool_response.content.copy()
        new_content[0]["text"] = f"🔥 {new_content[0]['text']}"
        return ToolResponse(
            content=new_content,
            metadata=tool_response.metadata
        )
    return tool_response

# 注册工具时设置预设参数和后处理函数
toolkit.register_tool_function(
    tool_func=send_game_message,
    group_name="werewolf",
    preset_kwargs={"channel": "werewolf_game"},  # 预设频道
    postprocess_func=message_postprocessor
)

print("注册带预设参数的工具完成")

注册带预设参数的工具完成


In [8]:
## 五、内置文件操作工具示例

from agentscope.tool import view_text_file, write_text_file, insert_text_file

# 先创建文件操作工具组
create_tool_group_if_not_exists(
    toolkit,
    group_name="file_ops",
    description="文件操作工具",
    active=True,
    notes="使用文件工具时请注意文件路径的正确性，避免覆盖重要文件"
)

# 然后注册文件操作工具
toolkit.register_tool_function(view_text_file, group_name="file_ops")
toolkit.register_tool_function(write_text_file, group_name="file_ops") 
toolkit.register_tool_function(insert_text_file, group_name="file_ops")

# 创建测试文件
test_file_path = "/tmp/werewolf_game_log.txt"
initial_content = """狼人杀游戏日志
================
游戏开始时间: 2024-10-02
玩家列表:
1. Alice - 村民
2. Bob - 狼人
3. Charlie - 预言家
4. Diana - 女巫
"""

# 使用工具创建文件
write_result = await write_text_file(test_file_path, initial_content)
print("文件创建结果:", write_result.content[0]["text"])

# 查看文件内容
view_result = await view_text_file(test_file_path, ranges=[1, 5])
print("\n文件内容预览:")
print(view_result.content[0]["text"])

# 插入新内容
new_log_entry = "\n第一天:\n- 预言家查验Bob，发现是狼人\n"
insert_result = await insert_text_file(test_file_path, new_log_entry, 9)
print("\n插入内容结果:", insert_result.content[0]["text"])

✓ 创建工具组: file_ops
文件创建结果: Create and write /tmp/werewolf_game_log.txt successfully.

文件内容预览:
The content of /tmp/werewolf_game_log.txt in [1, 5] lines:
```
1: 狼人杀游戏日志
2: ================
3: 游戏开始时间: 2024-10-02
4: 玩家列表:
5: 1. Alice - 村民
```

插入内容结果: Insert content into /tmp/werewolf_game_log.txt at line 9 successfully. The new content between lines 4-12 is:
```
4: 玩家列表:
5: 1. Alice - 村民
6: 2. Bob - 狼人
7: 3. Charlie - 预言家
8: 4. Diana - 女巫
9: 
10: 
11: 第一天:
12: - 预言家查验Bob，发现是狼人
```


In [10]:
## 六、DashScope 多媒体工具示例

from agentscope.tool import dashscope_text_to_image, dashscope_text_to_audio, dashscope_image_to_text

# 先创建多媒体工具组
create_tool_group_if_not_exists(
    toolkit,
    group_name="multimedia",
    description="多媒体生成和处理工具",
    active=True,
    notes="使用多媒体工具需要有效的DashScope API密钥"
)

# 然后注册DashScope工具
toolkit.register_tool_function(dashscope_text_to_image, group_name="multimedia")
toolkit.register_tool_function(dashscope_text_to_audio, group_name="multimedia")
toolkit.register_tool_function(dashscope_image_to_text, group_name="multimedia")

# 文本转图片示例
print("=== 文本转图片示例 ===")
image_prompt = "一个神秘的狼人杀游戏场景，圆桌周围坐着戴面具的玩家"
try:
    image_result = dashscope_text_to_image(
        prompt=image_prompt,
        api_key=os.environ["DASHSCOPE_API_KEY"],
        n=1,
        size="1024*1024"
    )
    print("图片生成结果:", image_result.content[0]["text"])
except Exception as e:
    print(f"图片生成失败: {e}")

# 文本转语音示例
print("\n=== 文本转语音示例 ===")
audio_text = "欢迎来到狼人杀游戏！请所有玩家准备就绪。"
try:
    audio_result = dashscope_text_to_audio(
        text=audio_text,
        api_key=os.environ["DASHSCOPE_API_KEY"],
        model="sambert-zhichu-v1"
    )
    print("语音生成结果:", audio_result.content[0]["text"])
except Exception as e:
    print(f"语音生成失败: {e}")

✓ 创建工具组: multimedia
=== 文本转图片示例 ===
图片生成失败: 'text'

=== 文本转语音示例 ===
语音生成失败: 'text'


In [13]:
## 七、工具状态管理

# 保存工具包状态
print("=== 工具包状态管理 ===")
toolkit_state = toolkit.state_dict()
print("当前工具包状态:", toolkit_state)

# 清空工具包
print("\n清空工具包...")
toolkit.clear()
print("清空后的工具数量:", len(toolkit.get_json_schemas()))

# 恢复工具包状态
print("\n恢复工具包状态...")
# 需要重新创建工具组和注册工具函数，因为clear()会删除所有内容

# 先重新创建必要的工具组
create_tool_group_if_not_exists(
    toolkit,
    group_name="math",
    description="数学计算相关工具",
    active=True,
    notes="使用数学工具时请确保输入参数的有效性"
)

# 然后重新注册工具函数
toolkit.register_tool_function(execute_python_code)
toolkit.register_tool_function(calculate_fibonacci, group_name="math")

# 加载状态（主要是激活的工具组）
toolkit.load_state_dict(toolkit_state)
print("恢复后的激活notes:", toolkit.get_activated_notes())

# 展示完整的状态恢复过程
print("\n=== 状态恢复演示 ===")
print("1. 保存状态 -> 2. 清空工具包 -> 3. 重新创建工具组 -> 4. 重新注册工具 -> 5. 加载状态")
print(f"最终工具数量: {len(toolkit.get_json_schemas())}")
print("最终工具列表:")
for schema in toolkit.get_json_schemas():
    print(f"- {schema['function']['name']}")

=== 工具包状态管理 ===
当前工具包状态: {'active_groups': []}

清空工具包...
清空后的工具数量: 0

恢复工具包状态...
✓ 创建工具组: math
恢复后的激活notes: 

=== 状态恢复演示 ===
1. 保存状态 -> 2. 清空工具包 -> 3. 重新创建工具组 -> 4. 重新注册工具 -> 5. 加载状态
最终工具数量: 1
最终工具列表:
- execute_python_code


In [14]:
## 八、综合示例：狼人杀游戏工具集

# 创建专门的狼人杀游戏工具包
werewolf_toolkit = Toolkit()

# 定义狼人杀游戏专用工具
async def start_game(player_count: int = 8) -> ToolResponse:
    """开始狼人杀游戏"""
    return ToolResponse(
        content=[{"type": "text", "text": f"狼人杀游戏开始！共{player_count}名玩家参与。"}],
        metadata={"action": "start_game", "player_count": player_count}
    )

async def night_action(player_name: str, action: str, target: str = None) -> ToolResponse:
    """执行夜晚行动"""
    if target:
        message = f"{player_name}在夜晚执行了{action}，目标是{target}"
    else:
        message = f"{player_name}在夜晚执行了{action}"
    
    return ToolResponse(
        content=[{"type": "text", "text": message}],
        metadata={"player": player_name, "action": action, "target": target, "phase": "night"}
    )

async def day_vote(voter: str, target: str) -> ToolResponse:
    """白天投票"""
    return ToolResponse(
        content=[{"type": "text", "text": f"{voter}投票给{target}"}],
        metadata={"voter": voter, "target": target, "phase": "day"}
    )

# 先创建工具组
werewolf_toolkit.create_tool_group("game_control", "游戏控制工具", active=True, 
                                   notes="用于控制游戏流程的工具")
werewolf_toolkit.create_tool_group("night_phase", "夜晚阶段工具", active=False,
                                   notes="仅在夜晚阶段使用")
werewolf_toolkit.create_tool_group("day_phase", "白天阶段工具", active=True,
                                   notes="用于白天讨论和投票")
werewolf_toolkit.create_tool_group("utility", "实用工具", active=True,
                                   notes="辅助工具，如代码执行等")

# 然后注册狼人杀工具
werewolf_toolkit.register_tool_function(start_game, group_name="game_control")
werewolf_toolkit.register_tool_function(night_action, group_name="night_phase")
werewolf_toolkit.register_tool_function(day_vote, group_name="day_phase")
werewolf_toolkit.register_tool_function(execute_python_code, group_name="utility")

print("狼人杀工具包创建完成!")
print(f"已注册工具数量: {len(werewolf_toolkit.get_json_schemas())}")
print(f"激活的工具组: {werewolf_toolkit.get_activated_notes()}")

# 创建狼人杀游戏主持人
game_master = ReActAgent(
    name="GameMaster",
    sys_prompt=f"""你是狼人杀游戏的主持人。
    
可用工具说明：
{werewolf_toolkit.get_activated_notes()}

请根据游戏阶段合理使用工具来主持游戏。""",
    model=DashScopeChatModel(
        model_name="qwen-max",
        api_key=os.environ["DASHSCOPE_API_KEY"],
        stream=True,
        enable_thinking=True,
    ),
    formatter=DashScopeChatFormatter(),
    toolkit=werewolf_toolkit,
)

# 测试游戏主持人
test_msg = Msg(
    name="player",
    content="请开始一个8人局的狼人杀游戏",
    role="user"
)

print("\n=== 狼人杀游戏主持人测试 ===")
response = await game_master(test_msg)
print(f"游戏主持人: {response.content}")

狼人杀工具包创建完成!
已注册工具数量: 3
激活的工具组: ## About game_control Tools
用于控制游戏流程的工具
## About day_phase Tools
用于白天讨论和投票
## About utility Tools
辅助工具，如代码执行等

=== 狼人杀游戏主持人测试 ===
GameMaster: {
    "type": "tool_use",
    "id": "call_ea52916f850746c78fd6d4",
    "name": "start_game",
    "input": {
        "player_count": 8
    }
}
system: {
    "type": "tool_result",
    "id": "call_ea52916f850746c78fd6d4",
    "name": "start_game",
    "output": [
        {
            "type": "text",
            "text": "狼人杀游戏开始！共8名玩家参与。"
        }
    ]
}
GameMaster: 狼人杀游戏开始！我们总共有8名玩家参与。接下来我会分配身份，请各位玩家确认自己的身份并且记住它。身份分配完成后，我们将进入夜晚阶段。现在请闭眼，我将私聊给每位玩家分配身份。
游戏主持人: 狼人杀游戏开始！我们总共有8名玩家参与。接下来我会分配身份，请各位玩家确认自己的身份并且记住它。身份分配完成后，我们将进入夜晚阶段。现在请闭眼，我将私聊给每位玩家分配身份。
